# Continuous Optimisation HW1

In [2]:
# Imports
import numpy as np
import scipy

# Question 1
Implement phi(x, P), bigphi(X, P), f(X, P, y).

In [13]:
# Question 1
#TODO Vectorise all the functions


def h(x: np.array):
    """Gaussian filter"""
    return np.e**(-np.inner(x, x)/0.01)  # Always take sigma = 0.1


def phi(x, P):
    """Calculate contribution of each 'true' star to observed image"""
    phi = []
    for i in range(np.size(P)):
        phi.append(h(P[i]-x))
    return phi


def bigphi(X, P):
    """Calculate image observed, based on K-star positions X"""
    K = X.shape[1]  # Number of stars
    bigphi = []
    for i in range(K):
        xc  = np.array([X[0][i], X[1][i]])
        bigphi += phi(xc, P)
    return bigphi


def  f(X, P, y):
    """Calculate squared error of estimate bigphi(X)"""
    sizeP = int(np.size(P))
    return (1/(2*sizeP)) * np.linalg.norm(bigphi(X, P)-y)**2




# Question 2

# Question 3

In [ ]:
# Question 3